In [154]:
import scapy.all as sc
import numpy as np
import pandas as pd
from scipy.stats import entropy

In [320]:
SNIFF_TIMEOUT = 600
BROADCAST = "BROADCAST"
UNICAST = "UNICAST"
PCAP_FILE = "packages.pcap"
ARP_OP_WHO_HAS = 1
ARP_OP_IS_AT = 2

## Hago sniff durante sniff_timeout segundos

In [321]:
packages = sc.sniff(timeout=SNIFF_TIMEOUT)

## Guardo los paquetes en formato pcap

In [322]:
sc.wrpcap(PCAP_FILE,packages)

## Leo los paquetes del archivo .pcap

In [323]:
packages = sc.sniff(offline=PCAP_FILE)

## Ejercicio 3.2.1

In [324]:
def simbolo_s(package):
    if p.dst == 'ff:ff:ff:ff:ff:ff':
        return BROADCAST
    else:
        return UNICAST

In [325]:
def entropia_fuente(fuente):
    counts = fuente.value_counts()
    return entropy(base=2,pk=counts.as_matrix())

In [326]:
def informacion_simbolos(fuente):
    counts = fuente.value_counts()
    total = counts.sum()
    probs = counts / float(total)
    return pd.Series(- np.log2(probs), index=counts.index)

In [327]:
fuente = pd.Series([simbolo_s(p) for p in packages])
contadores = fuente.value_counts()
print "Entropia fuente S =", entropia_fuente(fuente)
print contadores
print informacion_simbolos(fuente)

Entropia fuente S = 0.250502744127
UNICAST      7564
BROADCAST     330
dtype: int64
UNICAST      0.061607
BROADCAST    4.580219
dtype: float64


## Ejercicio 3.2.2

In [328]:
arp_packages = sc.sniff(offline=PCAP_FILE, lfilter=lambda x: x.haslayer(sc.ARP))

In [329]:
arp_packages = map(lambda p: p["ARP"], arp_packages)
who_has_packages = filter(lambda p : p.op == ARP_OP_WHO_HAS,arp_packages)
is_at_packages = filter(lambda p : p.op == ARP_OP_IS_AT, arp_packages)

In [330]:
is_at_packages[0]

<ARP  hwtype=0x1 ptype=0x800 hwlen=6 plen=4 op=is-at hwsrc=e4:48:c7:c8:31:b9 psrc=192.168.1.254 hwdst=c4:85:08:d9:07:e2 pdst=192.168.1.100 |>

In [331]:
who_has_packages[0]

<ARP  hwtype=0x1 ptype=0x800 hwlen=6 plen=4 op=who-has hwsrc=5c:51:88:4d:6c:13 psrc=192.168.1.10 hwdst=00:00:00:00:00:00 pdst=192.168.1.1 |>

## Analisis de simbolos distinguidos utilizando who-has packages

In [332]:
def simbolo_s2(package):
    return package.pdst

In [333]:
fuente_s2 = pd.Series([simbolo_s2(p) for p in who_has_packages])
contadores = fuente_s2.value_counts()
print contadores
print "Entropia de la fuente =", entropia_fuente(fuente_s2)
print informacion_simbolos(fuente_s2)

192.168.1.1      61
192.168.1.10      3
192.168.1.254     2
dtype: int64
Entropia de la fuente = 0.460608284608
192.168.1.1      0.113657
192.168.1.10     4.459432
192.168.1.254    5.044394
dtype: float64


## Analisis de simbolos distinguidos utilizando is_at packages

In [334]:
def simbolo_s3(package):
    return package.psrc

In [335]:
fuente_s3 = pd.Series([simbolo_s3(p) for p in is_at_packages])
contadores_s3 = fuente_s3.value_counts()
print contadores_s3
print "Entropia de la fuente =", entropia_fuente(fuente_s3)
print informacion_simbolos(fuente_s3)

192.168.1.254    1
dtype: int64
Entropia de la fuente = -0.0
192.168.1.254   -0.0
dtype: float64


In [336]:
len(is_at_packages)

1